In [2]:
import requests
import json
import pandas as pd
from getpass import getpass
import folium
import os
from dotenv import load_dotenv
load_dotenv() 
token = os.getenv("token")
from pymongo import MongoClient
import time
client = MongoClient("localhost:27017")
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster



In [3]:
client
db = client["ironhack"]
c = db.get_collection("companies")
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [6]:
# Conditions to find technological companies in Canada
condition_1 = {'category_code': {"$in": ["software", "web", "games_video", "network_hosting"]}}
condition_2 = {"offices.country_code": "CAN"}
projection = {"name": 1, "_id": 0}


companies = {"$and": [condition_1, condition_2]}
count = c.count_documents(companies)

print(f"Number of companies that match the conditions: {count}")

Number of companies that match the conditions: 308


In [5]:
# Companies in Canada for each city Montreal

condition_1 = {'offices.city': "Montreal"}
condition_2 = {"offices.country_code": "CAN"}
projection = {"name": 1, "_id": 0}


companies = {"$and": [condition_1, condition_2]}
count = c.count_documents(companies)

print(f"Number of companies that match the conditions: {count}")

Number of companies that match the conditions: 48


In [6]:
# Companies in Canada for each city Toronto

condition_1 = {'offices.city': "Toronto"}
condition_2 = {"offices.country_code": "CAN"}
projection = {"name": 1, "_id": 0}


companies = {"$and": [condition_1, condition_2]}
count = c.count_documents(companies)

print(f"Number of companies that match the conditions: {count}")

Number of companies that match the conditions: 131


In [7]:
# Companies in Canada for each city Ottawa

condition_1 = {'offices.city': "Ottawa"}
condition_2 = {"offices.country_code": "CAN"}
projection = {"name": 1, "_id": 0}


companies = {"$and": [condition_1, condition_2]}
count = c.count_documents(companies)

print(f"Number of companies that match the conditions: {count}")

Number of companies that match the conditions: 24


In [68]:
# Function to get the collection companies into a DF

def get_companies_dataframe(collection, conditions, projection):
    # Use the find method to retrieve the matching documents
    companies = list(collection.find(conditions, projection))

    # Create a Pandas DataFrame from the list of documents
    df = pd.json_normalize(companies, "offices", ["name"])
    df = df.rename(columns={"latitude": "company_latitude", "longitude": "company_longitude"})

    # Drop rows with NaN values
    df = df.dropna()

    return df

In [69]:
# Getting a companies DF for Montreal

conditions = {"$and": [{"offices.city": "Montreal"}, {}]}
projection = {"name": 1, "_id": 0, "offices.latitude": 1, "offices.longitude": 1}

df_mo = get_companies_dataframe(c, conditions, projection)
df_mo

,company_latitude,company_longitude,name
0,37.421972,-122.084143,Google
1,42.280988,-83.748882,Google
2,33.781466,-84.387519,Google
3,30.351416,-97.751382,Google
4,40.021416,-105.260931,Google
...,...,...,...
146,45.496198,-73.676383,Trellia Networks
147,45.545447,-73.639076,Omniscient Technology
148,45.512433,-73.555016,Hopper
150,45.503971,-73.572360,mTrip


In [22]:
# Montreal Coordinates

lat_mo = 45.502033 
lon_mo = -73.622622 

In [8]:
#Crating a pin map for the city of Montreal

map_1 = Map(location = [lat_mo, lon_mo], zoom_start=11)

for index, row in df_mo.iterrows():
    
    
    lat_companies = row["company_latitude"]
    lon_companies = row["company_longitude"]

    new_marker = Marker(location = [lat_companies, lon_companies])
    
    new_marker.add_to(map_1)

map_1
map_1.save('../DATA/montreal_map.html')


NameError: name 'df_mo' is not defined

In [93]:
#Creating a function for making heat maps

def create_heatmap(dataframe, lat_column, lon_column, zoom_level, location):
    
    map_ = folium.Map(location=location, zoom_start=zoom_level)
    
    heat_map = HeatMap(data=dataframe[[lat_column, lon_column]])
    
    heat_map.add_to(map_)

    return map_




In [94]:
# Using function to create a Heat Map for Montreal

location = [lat_mo, lon_mo]
zoom_level = 11
map_2 = create_heatmap(df_mo, "company_latitude", "company_longitude", zoom_level, location)
map_2

map_2.save('../DATA/montreal_heat_map.html')

In [43]:
# TORONTO latitud and longitud

tor_lat = 43.719256 
tor_lon = -79.379482

In [72]:
# Getting a companies DF for Toronto

conditions = {"$and": [{"offices.city": "Toronto"}, {}]}
projection = {"name": 1, "_id": 0, "offices.latitude": 1, "offices.longitude": 1}

df_to = get_companies_dataframe(c, conditions, projection)
df_to

,company_latitude,company_longitude,name
0,37.421972,-122.084143,Google
1,42.280988,-83.748882,Google
2,33.781466,-84.387519,Google
3,30.351416,-97.751382,Google
4,40.021416,-105.260931,Google
...,...,...,...
246,43.707147,-79.396386,Cybernomics
248,43.653863,-79.367013,Stormfisher Biogas
250,43.670233,-79.386755,Constellation Software
252,43.646735,-79.402220,Streamlogics


In [55]:
#Creating map for Toronto
# Pin map for Toronto with all companies in the city

map_to = Map(location = [tor_lat, tor_lon], zoom_start=11)
for index, row in df_to.iterrows():
    
    # 1. location of the accident
    lat_accident = row["company_latitude"]
    lon_accident = row["company_longitude"]

    new_marker = Marker(location = [lat_accident, lon_accident])
    
    new_marker.add_to(map_to)

map_to
map_to.save('../DATA/toronto_map.html')

In [73]:
# Creating a companies Heat Map for Toronto

location = [tor_lat, tor_lon]
zoom_level = 11
map_3 = create_heatmap(df_to, "company_latitude", "company_longitude", zoom_level, location)
map_3
map_3.save('../DATA/toronto_heat_map.html')

In [60]:
# OTAWA coordinates

ott_lat = 45.409977
ott_lon = -75.701231

In [74]:
# Getting a companies DF for Ottawa

conditions = {"$and": [{"offices.city": "Ottawa"}, {}]}
projection = {"name": 1, "_id": 0, "offices.latitude": 1, "offices.longitude": 1}

df_ot = get_companies_dataframe(c, conditions, projection)
df_ot

,company_latitude,company_longitude,name
0,45.352255,-75.834151,Protecode
1,45.426940,-75.692196,Rove
2,45.427398,-75.689871,Axentra
4,40.746368,-73.983972,Xandros
5,50.980474,7.345009,Xandros
6,45.326530,-75.835706,Xandros
7,52.413065,16.827764,Xandros
9,-23.527594,-46.677049,Xandros
11,45.392766,-75.752398,Base 10 Web Solutions
12,45.420233,-75.701550,itteco


In [63]:
#Creating a map for Ottawa
# Getting a companies pin map

map_4 = Map(location = [ott_lat, ott_lon], zoom_start=13)

for index, row in df_ot.iterrows():
    
    # 1. location of the accident
    lat_accident = row["company_latitude"]
    lon_accident = row["company_longitude"]

    new_marker = Marker(location = [lat_accident, lon_accident])
    
    new_marker.add_to(map_4)

map_4
map_4.save('../DATA/ottawa_map.html')

In [96]:
# Creating a companies heat map for Ottawa

location = [ott_lat, ott_lon]
zoom_level = 11
map_5 = create_heatmap(df_ot, "company_latitude", "company_longitude", zoom_level, location)
map_5
map_5.save('../DATA/toronto_map.html')